In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
import datetime
import os,gc
import utils

file_path='D:/360WiFi/'
raw_data_path='D:/360WiFi/FDDC_financial_data_20180613/'
split_file_path='D:/360WiFi/files/financial_data_new/'
input_path='D:/360WiFi/files/input/'
mid_path='D:/360WiFi/files/mid_file/'
other_path='D:/360WiFi/files/other_file/'

'''
generate_data_base() 执行所有函数，生成table_base 表

'''

start_time='2009-01-01'


def tick_symbol_map():
    '''
    生成ticker_symbol_2006/2008文件
    '''
    
    h=pd.read_csv(other_path+'level_1_3_industry_class_shenwan.csv',encoding='gbk')
    h=h[['ticker','industryID','industrySymbol','intoDate','outDate','isNew']]
    h['outDate']=h['outDate'].fillna('2018/3/30')

    h['intoDate']=pd.to_datetime(h['intoDate'],format='%Y/%m/%d') 
    h['outDate']=pd.to_datetime(h['outDate'],format='%Y/%m/%d') 
    h=h[h['outDate']>='2006-01-01']
    h=h.sort_values(by=['ticker','intoDate'])
    h['intoDate']=h['intoDate'].apply(lambda x:datetime.datetime.strftime(x,format='%Y/%m/%d') )
    h['outDate']=h['outDate'].apply(lambda x:datetime.datetime.strftime(x,format='%Y/%m/%d') )
    h
    month_list=[]
    ticker_list=[]
    symbol_list=[]
    id_list=[]
    for indexs in h.index:
        month=getBetweenMonth(h.loc[indexs,'intoDate'],h.loc[indexs,'outDate'])
        month_list.extend(month)
        ticker=h.loc[indexs,'ticker']
        ticker_list.extend([ticker for x in range(len(month))])
        symbol=h.loc[indexs,'industrySymbol']
        symbol_list.extend([symbol for x in range(len(month))])
        id_=h.loc[indexs,'industryID']
        id_list.extend([id_ for x in range(len(month))])

    print(len(month_list))
    print(len(ticker_list))
    print(len(symbol_list))
    print(len(id_list))
    id_symbol=DataFrame({'TICKER_SYMBOL':ticker_list,'time':month_list,'industrySymbol_level_3':symbol_list,'industryID':id_list})
    id_symbol.drop_duplicates(['TICKER_SYMBOL','time'], keep='last',inplace=True)
    return id_symbol


def merge_base():
    '''
    基于Market Data和ticker_symbol_2006，生成merge_base基准表，
    
    '''
    #market_data=pd.read_csv(file_path+'/files/Market Data_20180529.csv',encoding='gbk')
    market_data=pd.read_excel(other_path+'[New] Market Data_20180613.xlsx',encoding='gbk')
    ticker_symbol=pd.read_csv(other_path6+'ticker_symbol_2006.csv',encoding='gbk')
    market_data['END_DATE_']=pd.to_datetime(market_data['END_DATE_'],format='%Y/%m/%d') 
    market_data=market_data[['TICKER_SYMBOL','END_DATE_','TYPE_ID']]
    market_data.rename(columns={'TYPE_ID':'industryID_level_3'},inplace=True)


    market_data['year']=market_data['END_DATE_'].dt.year
    market_data['month']=market_data['END_DATE_'].dt.month

    market_data=market_data[market_data['END_DATE_']>='2008-01-01']
    ticker_symbol['year']=ticker_symbol['time'].map(lambda x:'%s'%str(x)[0:4]).astype(int)
    ticker_symbol['month']=ticker_symbol['time'].map(lambda x:'%s'%str(x)[4:]).astype(int)
    ticker_symbol=ticker_symbol[ticker_symbol['TICKER_SYMBOL']!='DY600018']
    ticker_symbol['TICKER_SYMBOL']=ticker_symbol['TICKER_SYMBOL'].astype(int)

    ticker_symbol
    test=pd.merge(ticker_symbol,market_data,how='left',on=['TICKER_SYMBOL','year','month'])
    
    ##以market data中symbol为准，如果有不同，则将industryID替换为market data中的id
    index_=test[(test['industryID']!=test['industryID_level_3'])&(test['industryID_level_3'].isnull().values==False)].index#找出两列不相等值的地方
    #test[(test['industryID_level_3'].isnull().values==False)]#选出某列不为空的记录
    test.loc[index_,'industryID']=test.loc[index_,'industryID_level_3']

    test.to_csv(file_path+'merge_base.csv',index=False)
    
    return 


##生成 Bigtable表
def gengerate_big_table():
    '''
    merge_base：包含公司从上市以来每个月对应的industry_ID,
    ID_industry_index_Symbol:industry_ID对应的一二三级industry_symbol
    
    以上两者merge后，生成ticker_industry_record.csv,
    
    再和mergefile（大表）merge生成Bigtable
    '''

    base=pd.read_csv(mid_path+'merge_base.csv',encoding='gbk')
    base.drop(['industrySymbol_level_3','time','END_DATE_','industryID_level_3'],axis=1,inplace=True)
    base.rename(columns={'industryID':'industryID_level_3'},inplace=True)


    id_symbol=pd.read_csv(other_path+'ID_industry_index_Symbol.csv',encoding='gbk')
    columns=['industrySymbol_level_1', 'industrySymbol_level_2', 'industrySymbol_level_3', 'indexSymbol_level_1',\
            'indexSymbol_level_2','indexSymbol_level_3']
    for item in columns:
        id_symbol[item]=id_symbol[item].fillna(0).astype(int)

    tick_symbol_index=pd.merge(base,id_symbol,on=['industryID_level_3'])
    
    tick_symbol_index.to_csv(other_path+'ticker_industry_record.csv',index=False,encoding='gbk')
    ###tick_symbol_index[tick_symbol_index['TICKER_SYMBOL']==600832].sort_values(by=['year','month'])

    Table=pd.read_csv(mid_path+'merge_file.csv',encoding='gbk')

    Table['END_DATE']=pd.to_datetime(Table['END_DATE'],format='%Y/%m/%d') 

    Table['year']=Table['END_DATE'].dt.year
    Table['month']=Table['END_DATE'].dt.month


    Bigtable=pd.merge(Table,tick_symbol_index,how='left',on=['TICKER_SYMBOL','year','month'])
    Bigtable.drop(['year', 'month','level_1','level_2'],axis=1,inplace=True)#删除tick_symbol_index中一些无用特征


    Bigtable[['industryID_level_3', 'industryID_level_1', 'industryName_level_1',
           'industrySymbol_level_1', 'indexSymbol_level_1', 'industryID_level_2',
           'industryName_level_2', 'industrySymbol_level_2', 'indexSymbol_level_2',
           'industryName_level_3', 'industrySymbol_level_3', 'indexSymbol_level_3',
           'isNew']]

    columns=['industrySymbol_level_1','indexSymbol_level_1','industrySymbol_level_2',\
             'indexSymbol_level_2','industrySymbol_level_3', 'indexSymbol_level_3']
    for item in columns:
        Bigtable[item]=Bigtable[item].fillna(0).astype(int)
    
    return Bigtable


class Table_base(object):
    
    '''
    在Bigtable基础上，添加class_by_14L1,2,3三列
    '''
    def __init__(self,table):
        self.Bigtable=table
        pass
        
    '''
    财报信息早于股市信息，所以有一些在财报发布年份，公司的具体行业代码尚不清楚。其中一部分可以根据level1_3完善(ticker_symbol)去解决，正在想办法。还有一部分暂时解决不了。
    '''
    #@staticmethod 
    def L1_classifi(self):
        '''
        一级分类操作
        '''
        print('adding 1st level classify columns ')
        Bigtable=self.Bigtable
        Bigtable['Mid_L2']=Bigtable['industrySymbol_level_2']
        Bigtable['Mid_L3']=Bigtable['industrySymbol_level_3']
        Bigtable['classifi_by_14L1']=Bigtable['industrySymbol_level_1']

        index_1=Bigtable[(Bigtable['Mid_L2']==340200)|(Bigtable['Mid_L2']==340100)].index
        index_1
        Bigtable.loc[index_1,'Mid_L2']=Bigtable.loc[index_1,'Mid_L2'].map({340200:340400,340100:340400})
        class_map_L1=pd.read_csv(file_path+'/files/value_map/classifi_by_14L1.csv')
        class_map_L1
        dic_14L1=dict(zip(class_map_L1.indusSymbol_L23_11,class_map_L1.indusSymbol_L1_14))
        dic_14L1
        for (a,b) in dic_14L1.items():
            index_2=Bigtable[(Bigtable['Mid_L2']==a)|(Bigtable['Mid_L3']==a)].index
            Bigtable.loc[index_2,'classifi_by_14L1']=b

        Bigtable[(Bigtable['Mid_L2']==250100)|(Bigtable['Mid_L3']==310307)]
        Bigtable.classifi_by_14L1.unique()
        Bigtable[Bigtable['classifi_by_14L1']==0]
        
        return Bigtable

    #@staticmethod 
    def map_08_11(self):
        '''
        实现08-11年的代码转换
        '''
        
        Bigtable=self.Bigtable
        #08-11一般替换
        map_0811_basic=pd.read_csv(file_path+'/files/value_map/map_08_11_basic.csv',encoding='gbk')
        map_0811_basic_dic=dict(zip(map_0811_basic['08_L3_symbol'],zip(map_0811_basic['11_L3_symbol'],map_0811_basic['11_L2_symbol'])))
        map_0811_basic_dic.items()
        for (a,(b,c)) in map_0811_basic_dic.items():
            index_2=Bigtable[Bigtable['Mid_L3']==a].index
            Bigtable.loc[index_2,'Mid_L3']=b
            Bigtable.loc[index_2,'Mid_L2']=c

        #08-11特殊替换
        map_0811_specific=pd.read_csv(file_path+'/files/value_map/map_08_11_specific.csv',encoding='gbk')
        map_0811_specific['ticker']=map_0811_specific['ticker'].fillna(0).astype(int)
        map_0811_specific
        map_0811_specific_dic=dict(zip(map_0811_specific['ticker'],zip(map_0811_specific['08_L3_symbol'],\
                                    zip(map_0811_specific['11_L3_symbol'],map_0811_specific['11_L2_symbol']))))
        for (a,(b,(c,d))) in map_0811_specific_dic.items():
            #sample:2263 220501 220501 220500
            #print(a,b,c,d)
            index_2=Bigtable[(Bigtable['END_DATE']<'2011-10-10')&(Bigtable['TICKER_SYMBOL']==a)&(Bigtable['Mid_L3']==b)].index
            Bigtable.loc[index_2,'Mid_L3']=c
            Bigtable.loc[index_2,'Mid_L2']=d

        return Bigtable

    #@staticmethod 
    def L2_classifi(self):
        '''
        二级分类操作
        '''
        print('adding 2nd level classify columns ')
        Bigtable=self.L1_classifi()
        Bigtable['Mid_L2']=Bigtable['industrySymbol_level_2']
        Bigtable['Mid_L3']=Bigtable['industrySymbol_level_3']
        Bigtable['classifi_by_14L2']=Bigtable['industrySymbol_level_2']

        Bigtable=self.map_08_11()

        #一般转换11-14
        class_map_L2_basic=pd.read_csv(file_path+'/files/value_map/classifi_by_14L2_basic.csv',encoding='gbk')
        class_map_L2_basic
        dic_14L2_basic=dict(zip(class_map_L2_basic.indusSymbol_L3_11,class_map_L2_basic.indusSymbol_L2_14))
        dic_14L2_basic.items()
        for (a,b) in dic_14L2_basic.items():
            index_2=Bigtable[(Bigtable['END_DATE']<='2014-01-01')&(Bigtable['Mid_L3']==a)].index
            Bigtable.loc[index_2,'classifi_by_14L2']=b


        #特殊转换11-14
        class_map_L2_specific=pd.read_csv(file_path+'/files/value_map/classifi_by_14L2_specific.csv',encoding='gbk')
        class_map_L2_specific
        dic_14L2_specific=dict(zip(class_map_L2_specific.ticker,zip(class_map_L2_specific.indusSymbol_L3_11,class_map_L2_specific.indusSymbol_L2_14)))
        dic_14L2_specific.items()
        for (a,(b,c)) in dic_14L2_specific.items():
            #sample:2081 250202 620200
            #print(a,b,c)
            index_2=Bigtable[(Bigtable['END_DATE']<='2014-01-01')&(Bigtable['TICKER_SYMBOL']==a)&(Bigtable['Mid_L3']==b)].index
            Bigtable.loc[index_2,'classifi_by_14L2']=c
        return Bigtable

    #@staticmethod 
    def L3_classifi(self):

        ##三级分类操作
        print('adding 3rd level classify columns ')
        Bigtable=self.L2_classifi()
        Bigtable['Mid_L2']=Bigtable['industrySymbol_level_2']
        Bigtable['Mid_L3']=Bigtable['industrySymbol_level_3']
        Bigtable['classifi_by_14L3']=Bigtable['industrySymbol_level_3']

        Bigtable=self.map_08_11()

        #一般转换11-14
        class_map_L3_basic=pd.read_csv(file_path+'/files/value_map/classifi_by_14L3_basic.csv',encoding='gbk')
        class_map_L3_basic
        dic_14L3_basic=dict(zip(class_map_L3_basic.indusSymbol_L3_11,class_map_L3_basic.indusSymbol_L3_14))
        dic_14L3_basic.items()
        for (a,b) in dic_14L3_basic.items():
            #print(a,b)
            index_2=Bigtable[(Bigtable['END_DATE']<='2014-01-01')&(Bigtable['Mid_L3']==a)].index
            Bigtable.loc[index_2,'classifi_by_14L3']=b


        #特殊转换11-14
        class_map_L3_specific=pd.read_csv(file_path+'/files/value_map/classifi_by_14L3_specific.csv',encoding='gbk')
        #先处理一下空格
        class_map_L3_specific['ticker']=class_map_L3_specific['ticker'].apply(lambda x: 0 if x.isspace() else x)
        class_map_L3_specific['ticker']=class_map_L3_specific['ticker'].astype(int)

        dic_14L3_specific=zip(class_map_L3_specific.indusSymbol_L3_11,zip(class_map_L3_specific.ticker,class_map_L3_specific.indusSymbol_L3_14))
        dic_14L3_specific

        for (a,(b,c)) in dic_14L3_specific:
            #sample:260404 2350 630402
            #print(a,b,c)
            index_2=Bigtable[(Bigtable['END_DATE']<='2014-01-01')&(Bigtable['TICKER_SYMBOL']==b)&(Bigtable['Mid_L3']==a)].index
            Bigtable.loc[index_2,'classifi_by_14L3']=c
 

        return  Bigtable



def drop_rows(T):
    print('del some end_date before list_date...')
    early_date=pd.read_csv(other_path+'ticker_market.csv',encoding='gbk')
    early_date_dic=zip(early_date.TICKER_SYMBOL,early_date.listDate)

    T['END_DATE']=T['END_DATE'].apply(lambda x:datetime.datetime.strftime(x,format='%Y-%m-%d'))
    concat_list=[]
    for (a,b) in early_date_dic:
        concat_list.append(T[(T.TICKER_SYMBOL==a)&(T.END_DATE>=b)])

    T_new=pd.concat(concat_list)
    T_new['END_DATE']=pd.to_datetime(T_new['END_DATE'],format='%Y/%m/%d') 
    T_new=T_new[T_new['END_DATE']>=start_time]
    
    return T_new

def generate_data_base():
    table=gengerate_big_table()
    T=Table_base(table).L3_classifi()
    TT=drop_rows(T)
    TT.to_csv(mid_path+'table_base.csv',index=False)
    return TT